# Training models: validation, selection, inspection

In [ ]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx
PATH_YIELD_ENGINE = 'src'
def set_paths() -> None:
    import sys
    import os
    
    if 'cwd' not in globals():
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
        os.chdir(cwd)   
    print(f"working dir is '{os.getcwd()}'")                            
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    print(f"added `{sys.path[0]}` to python paths")
set_paths()

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from pandas.api.types import is_numeric_dtype

from gamma.model.fitcv import RegressorFitCV
from gamma.model.inspection import RegressionModelInspector
from gamma.model.validation import CircularCrossValidator
from gamma import Sample
from gamma.model.selection import ModelParameterGrid, ModelRanker, summary_report
from gamma.sklearndf.pipeline import PipelineDF,RegressionPipelineDF 
from gamma.sklearndf.transformation import SimpleImputerDF, OneHotEncoderDF, ColumnTransformerDF
from gamma.sklearndf.transformation.extra import BorutaDF
from gamma.sklearndf.regression import RandomForestRegressorDF, LGBMRegressorDF
from gamma.sklearndf.classification import RandomForestClassifierDF

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
df = pd.read_csv('data/ames-housing-dataset/train.csv')
df = df.drop(['Id', 'YrSold', 'MoSold'], axis=1)
TARGET = "SalePrice"
sample = Sample(observations=df, target_name=TARGET)

In [ ]:
categorical_features = sample.features.select_dtypes([object]).columns
numerical_features = [col for col, dtype in sample.features.dtypes.iteritems() if is_numeric_dtype(dtype)]

## ModelPipeline

In [ ]:
ohe_df = PipelineDF(steps=
 [('imputer', SimpleImputerDF(strategy='constant', fill_value='nan')), 
  ('ohe', OneHotEncoderDF(sparse=False, handle_unknown='ignore'))])

imputer_df = SimpleImputerDF(strategy="median")

preprocessing_df = ColumnTransformerDF(transformers=
  [('categorical', ohe_df, categorical_features),('numerical', imputer_df, numerical_features)])

In [ ]:
predictor = RandomForestRegressorDF()
regression_pipeline_df = RegressionPipelineDF(regressor=predictor, preprocessing=preprocessing_df)

In [ ]:
regression_pipeline_df.fit(sample.features, sample.target)
regression_pipeline_df.score(sample.features, sample.target)

## Cross-validation

In [ ]:
cv = CircularCrossValidator(test_ratio=1/3, num_splits=6)

## Model and hyperparameter optimization

In [ ]:
random_forest_pipeline = RegressionPipelineDF(regressor=RandomForestRegressorDF(), preprocessing=preprocessing_df)
lgbm_pipeline = RegressionPipelineDF(regressor=LGBMRegressorDF(), preprocessing=preprocessing_df)

In [ ]:
grids = [ModelParameterGrid(pipeline=random_forest_pipeline, estimator_parameters={"n_estimators": [10, 20]}),
         ModelParameterGrid(pipeline=lgbm_pipeline, estimator_parameters={"learning_rate": [0.1, 0.2]})]

In [ ]:
ranker = ModelRanker(grids=grids, cv=cv)
ranking = ranker.run(sample, n_jobs=-3)
print(summary_report(ranking))

In [ ]:
top_model = ranking[0].model

## Predictor

In [ ]:
regressor_fit = RegressorFitCV(pipeline=top_model, cv=cv, sample=sample)

We can easily make predictions using the model trained with a given cross-validation split:

In [ ]:
predictions0 = regressor_fit.predictions_for_split(split_id=0)

In [ ]:
predictions0.head()

If necessary we can retrieve the traind model for a given training split itself: 

In [ ]:
model0 = regressor_fit[0]

Or we can even even retrieve a generator of all the model for all the training splits:

In [ ]:
models = [m for m in regressor_fit]

## Model inspection

In [ ]:
inspector = RegressionModelInspector(regressor_fit)

In [ ]:
inspector.feature_importances().head()

In [ ]:
sample.features.head()